# Students@SC Boardgame Tournament Ranker

This Jupyter Notebook will calculate the final rankings for the Students@SC Boardgame Tournament based on the provided configuration and score files. Most of the code powering this notebook can be found in the `boardgame_tournament_scoring` directory.

### Configuration File Specification

The configuration file is a TOML file that provides relevant data on each game in the tournament. At the top of the file, there must be a `config_id` field. This is simply used as a unique ID for the file. Each game in the file must have the following format:

```toml
[game_name]
scheme = ...
max_ranking_points = ...
number_of_scores = ...
num_player_range = ...
```

The fields for each game are as follows:
* `game_name`: the name (i.e., unique identifier) of the game
* `scheme`: the scoring scheme to use for the game. Valid options are
  * `"num_player_no_placing"`: scheme that normalizes continuous scores
  * `"num_player_placing"`: scheme that normalizes discrete, placing scores (e.g., 1st, 2nd, 3rd)
  * `"top_three"`: scheme that gives everyone 1 point, but gives more points to people coming in the top 3 for a game
* `max_ranking_points`: the maximum number of points that this game can contribute towards a contestant's overall score. This does not matter for `"top_three"`.
* `number_of_scores`: the number of scores that this game will contribute towards a contestant's overall score
* `num_player_range`: a list of two numbers (e.g., `[#, #]`) specifying the minimum and maximum number of people how can play in a single session of the game. This does not matter for `"top_three"`.

### Score File Specification

The score file is a CSV file containing information about the results of games played. Each row in this file represents a single contestant's result for a single instance of a game. There are four columns in this file:
1. The `name` column, which specifies the contestant's name
2. The `game` column, which specifies the game the contestant played
3. The `score` column, which specifies the contestant's score/placement in the game
4. The `num_players` column, which specifies the number of players in the game

The only other requirement for the score file is that entries in the `game` column **must** match one of the `game_name` fields in the configuration file. If an entry does not match one of these fields, the row will be treated as if it represents an unknown game, and, thus, it will be dropped.

## Specify Configuration and Score Files

In [ ]:
tournament_config_file = "./test.toml"
score_file = "./test.csv"

## Get Game Information from Configuration File

In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath("./boardgame_tournament_scoring/"))

In [ ]:
import pandas as pd

In [ ]:
from boardgame_tournament_scoring import (
    import_games_from_toml,
    split_score_df_by_game,
    get_max_scores_no_placing,
    get_normalized_scores,
    get_final_counted_scores_df,
    get_num_games_played,
)

In [ ]:
games = import_games_from_toml(tournament_config_file)

## Read the Score File

In [ ]:
score_df = pd.read_csv(score_file)

## Split Score Data by Game

In [ ]:
per_game_dfs = split_score_df_by_game(score_df, games)

## Get the Max Scores for Non-Placement Games

In [ ]:
max_scores = get_max_scores_no_placing(per_game_dfs, games)

## Normalize the Scores based on Max Ranking Points

In [ ]:
normed_per_game_dfs = get_normalized_scores(per_game_dfs, games, max_scores)

## Collect Scores that Should Be Counted Towards Result

In [ ]:
full_counted_df = get_final_counted_scores_df(normed_per_game_dfs, games)

## Get Number of Games Played for Each Player

In [ ]:
games_played_count = get_num_games_played(score_df)

# Winner: Overall Score

In [ ]:
perf_score_series = full_counted_df.sum(axis=1)
overall_score_dict = []
for name in perf_score_series.index.unique().tolist():
    person_score_dict = {}
    person_score_dict["name"] = name
    person_score_dict["score"] = perf_score_series[name] + games_played_count[name]
    overall_score_dict.append(person_score_dict)
overall_score_df = pd.DataFrame(data=overall_score_dict)
overall_score_df.sort_values(by="score", ascending=False)
display(overall_score_df)